In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
# import sys

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
%matplotlib notebook

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17046149457085610220
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15598606746
locality {
  bus_id: 1
}
incarnation: 5276446417967312130
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:05:00.0, compute capability: 7.0"
]


### Importing MNIST dataset

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Xavier Initialization 

In [3]:
def weight_variable (shape):
    initial = tf.truncated_normal(shape, stddev = np.sqrt(2.0/sum(shape)) )
    return tf.Variable(initial)

def bias_variable (shape):
#     initial = tf.constant(0.1, shape=shape)
    initial = tf.truncated_normal(shape, stddev = np.sqrt(1.0/sum(shape)) )
    return tf.Variable(initial)

### Create the model

In [4]:
# Create the model
x = tf.placeholder(tf.float32, [None, 784])
# W = tf.Variable(tf.zeros([784, 10]))
# b = tf.Variable(tf.zeros([10]))
# y = tf.matmul(x, W) + b

W_1 = weight_variable([784, 1024])
b_1 = bias_variable([1024])

W_2 = weight_variable([1024, 1024])
b_2 = bias_variable([1024])

W_3 = weight_variable([1024, 1024])
b_3 = bias_variable([1024])

W_4 = weight_variable([1024, 1024])
b_4 = bias_variable([1024])

W_5 = weight_variable([1024, 10])
b_5 = bias_variable([10])

y_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)
y_2 = tf.nn.relu(tf.matmul(y_1, W_2) + b_2)
y_3 = tf.nn.relu(tf.matmul(y_2, W_3) + b_3)
y_4 = tf.nn.relu(tf.matmul(y_3, W_4) + b_4)
y = tf.matmul(y_4, W_5) + b_5 # predicted

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10]) # original

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

### Run the model (q1, q2)

In [6]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Train
for _ in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(1000)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:",sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels}))

Accuracy: 0.9858


### Last Layer vizualization (q3)

In [11]:
y_est = tf.nn.softmax(y)
# ce = cross_entropy(sm)
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

print("Accuracy:",sess.run([accuracy, y_est], feed_dict={x: mnist.test.images[:10,],
                                      y_: mnist.test.labels[:10,]}))

Accuracy: [1.0, array([[2.93664184e-15, 3.35836109e-10, 1.25363240e-12, 1.01327289e-11,
        1.41688008e-15, 5.43817122e-16, 6.65411464e-18, 1.00000000e+00,
        4.32307236e-15, 9.67879665e-11],
       [5.13774336e-11, 4.48928503e-08, 1.00000000e+00, 8.86737125e-12,
        6.62747288e-12, 3.87197259e-14, 5.07285325e-11, 1.87416714e-11,
        9.21176004e-13, 8.93662852e-16],
       [2.06583972e-10, 9.99999881e-01, 1.18768977e-08, 5.69905893e-11,
        3.44415341e-09, 6.27861374e-10, 6.60687505e-08, 4.08552125e-08,
        1.17789876e-08, 4.68842812e-11],
       [9.99978900e-01, 7.10923658e-08, 9.32221326e-07, 4.19039132e-08,
        2.16138702e-07, 2.90155864e-08, 1.85680201e-05, 6.28859482e-07,
        5.91680944e-08, 5.80850497e-07],
       [1.33250531e-08, 1.14696173e-08, 8.54855742e-08, 4.03700878e-10,
        9.99972701e-01, 1.78669080e-08, 3.28026566e-08, 4.60147866e-08,
        8.73844908e-09, 2.70214496e-05],
       [5.76918374e-12, 1.00000000e+00, 1.33617464e-10, 1.9